In [3]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd

In [4]:
n_properties = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM opa_properties_public
    """
).json()["rows"][0]["count"]

print(f"There are {n_properties} properties as of {datetime.now().isoformat()}")

There are 581417 properties as of 2021-10-22T02:47:41.486038


In [5]:
n_properties = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT * FROM opa_properties_public where mailing_address
    """
).json()["rows"][0]["count"]

print(f"There are {n_properties} properties as of {datetime.now().isoformat()}")

KeyError: 'rows'

In [6]:
category_code_descs = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT category_code_description, count(*) FROM opa_properties_public 
    group by category_code_description
    """
).json()["rows"]
pd.DataFrame(category_code_descs)

,category_code_description,count
0,Commercial,14052
1,Industrial,4355
2,Mixed Use,14388
3,Multi Family,42497
4,Single Family,461778
5,Vacant Land,44307
6,None,40


In [7]:
zonings = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT zoning, count(*) FROM opa_properties_public 
    group by zoning
    """
).json()["rows"]
pd.DataFrame(zonings)

,zoning,count
0,CA1,974
1,CA2,379
2,CMX1,5531
3,CMX2,19392
4,CMX2.5,4857
5,CMX3,7879
6,CMX4,4127
7,CMX5,4943
8,I1,794
9,I2,3957


In [8]:
n_owner_1 = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(owner_1)) FROM opa_properties_public
    """
).json()["rows"][0]["count"]

print(f"There are {n_owner_1} unique owner_1's as of {datetime.now().isoformat()}")

n_owner_2 = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(owner_2)) FROM opa_properties_public
    """
).json()["rows"][0]["count"]

print(f"There are {n_owner_2} unique owner_2's as of {datetime.now().isoformat()}")

There are 431504 unique owner_1's as of 2021-10-22T02:47:46.332284
There are 153153 unique owner_2's as of 2021-10-22T02:47:47.518305


In [9]:
requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT (
        SELECT count(*)
        FROM opa_properties_public
        where  location like '%/%'
    )  as n_properties_w_fractions,
    (
        SELECT count(*)
        FROM opa_properties_public
        where  location like '%/%' and not location like '%1/2%'
    ) as n_properties_w_non_half_fractions
    """
).json()["rows"]

[{'n_properties_w_fractions': 528, 'n_properties_w_non_half_fractions': 0}]

In [10]:
requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
        SELECT distinct(suffix)
        FROM opa_properties_public
    """
).json()["rows"]

[{'suffix': None},
 {'suffix': 'S'},
 {'suffix': '2'},
 {'suffix': 'E'},
 {'suffix': 'R'},
 {'suffix': 'A'},
 {'suffix': 'L'}]

In [30]:
opa_properties_sample = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT *, ST_Y(the_geom) AS lat, ST_X(the_geom) AS lng FROM opa_properties_public 
    WHERE market_value is not null
    ORDER by market_value desc limit 5
    """
).json()["rows"]

In [31]:
opa_properties_sample[0]

{'cartodb_id': 562168,
 'the_geom': '0101000020E610000064A42747B3CA52C0613568906EF34340',
 'the_geom_webmercator': '0101000020110F00009628416475EB5FC18AF4B303F9815241',
 'assessment_date': None,
 'basements': None,
 'beginning_point': 'SWC DARIEN ST            ',
 'book_and_page': '0199308',
 'building_code': 'JD0  ',
 'building_code_description': 'AMUSE STAD ETC MASONRY',
 'category_code': '4 ',
 'category_code_description': 'Commercial',
 'census_tract': '806',
 'central_air': None,
 'cross_reference': '884145500',
 'date_exterior_condition': '2017-03-30T00:00:00Z',
 'depth': 1632.48,
 'exempt_building': 430417640,
 'exempt_land': 23779760,
 'exterior_condition': '3',
 'fireplaces': 0,
 'frontage': 975,
 'fuel': None,
 'garage_spaces': 0,
 'garage_type': '0',
 'general_construction': 'A ',
 'geographic_ward': '39',
 'homestead_exemption': 0,
 'house_extension': None,
 'house_number': '1020 ',
 'interior_condition': '3',
 'location': '1020 PATTISON AVE',
 'mailing_address_1': '2600 CE

In [90]:
homestead_exempt_count = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(*)
    FROM opa_properties_public where homestead_exemption is not null and homestead_exemption > 0
    """
).json()["rows"][0]["count"]
pct_homestead_exempt = round(homestead_exempt_count / n_properties * 100, 2)
print(
    f"There are {homestead_exempt_count} properties with homestead exemptions ({pct_homestead_exempt}%)."
)


homestead_exempt_mailing_count = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT  count(*)
    FROM opa_properties_public where homestead_exemption is not null and homestead_exemption > 0 and (mailing_street is not null or mailing_address_1 is not null)
    """
).json()["rows"][0]["count"]
pct_homestead_mailing_exempt = round(
    homestead_exempt_mailing_count / homestead_exempt_count * 100, 2
)
print(
    f"There are {homestead_exempt_mailing_count} properties"
    f" with homestead exemptions but also mailing addresses ({pct_homestead_mailing_exempt}% of exemptions)."
)

homestead_exempt_mailing_results = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT parcel_number, location, unit, mailing_care_of, 
    mailing_address_1, mailing_address_2, mailing_street, mailing_city_state, homestead_exemption
    FROM opa_properties_public
    where homestead_exemption is not null and homestead_exemption > 0 and 
    (mailing_street is not null or mailing_address_1 is not null) limit 10
    """
).json()["rows"]
print("Example of some of these is below:")
pd.DataFrame(homestead_exempt_mailing_results)

There are 225854 properties with homestead exemptions (38.84%).
There are 13750 properties with homestead exemptions but also mailing addresses (6.09% of exemptions).
Example of some of these is below:


,parcel_number,location,unit,mailing_care_of,mailing_address_1,mailing_address_2,mailing_street,mailing_city_state,homestead_exemption
0,011005300,219 EARP ST,None,None,None,None,219-221 EARP STREET,PHILADELPHIA PA,45000
1,011023600,230 WILDER ST,None,None,None,None,907 WALNUT AVE,LANGHORNE PA,45000
2,011014700,221 GERRITT ST,None,APONIEWICZ GINA,None,None,34 W SOUTHAMPTON AVE,PHILADELPHIA PA,45000
3,011017800,236 GERRITT ST,None,None,None,None,14 HEADLEY DR,NEWTOWN PA,45000
4,011034300,227 GREENWICH ST,None,None,None,None,2527 S 2ND ST,PHILADELPHIA PA,45000
5,011036600,132 GREENWICH ST,None,None,None,None,136 DICKINSON ST,PHILADELPHIA PA,45000
6,011028580,120 DICKINSON ST,000000F,None,UNIT 1,None,120 F DICKINSON ST,PHILADELPHIA PA,45000
7,011038600,248 GREENWICH ST,None,SLAVIC MICKI,None,None,152 MERCY ST,PHILADELPHIA PA,45000
8,011053900,1315 S HOWARD ST,None,None,None,None,113 WHARTON STREET,PHILADELPHIA PA,45000
9,011063500,1419 S 2ND ST,None,None,None,None,128 TASKER ST,PHILADELPHIA PA,45000


In [139]:
homestead_exempt_amounts = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT homestead_exemption, count(*)
    FROM opa_properties_public where homestead_exemption is not null group by homestead_exemption 
    """
).json()["rows"]
homestead_df = pd.DataFrame(homestead_exempt_amounts).astype(int)

group_size = 5000
homestead_df["exemption_quantity"] = pd.cut(
    homestead_df["homestead_exemption"],
    range(0, homestead_df["homestead_exemption"].max() + group_size, group_size),
    right=False,
)
homestead_value_counts_series = homestead_df.groupby("exemption_quantity").sum()[
    "count"
]
print("# of homestead exemptions by $ amount")
homestead_value_counts_series[homestead_value_counts_series > 0].sort_index()

# of homestead exemptions by $ amount


exemption_quantity
[0, 5000)               332460
[5000, 10000)               20
[10000, 15000)             196
[15000, 20000)             140
[20000, 25000)            1560
[25000, 30000)            1128
[30000, 35000)            1834
[35000, 40000)            1660
[40000, 45000)             130
[45000, 50000)          219159
[630000, 635000)             1
[1035000, 1040000)           1
[1575000, 1580000)           1
[1710000, 1715000)           1
[1800000, 1805000)           2
[2205000, 2210000)           1
[3960000, 3965000)           1
[11610000, 11615000)         1
[14760000, 14765000)         1
[18000000, 18005000)         1
[19080000, 19085000)         1
Name: count, dtype: int64

In [48]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


mailing_address_properties = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
        SELECT opa.* FROM (
            SELECT count(*) as num_properties, mailing_street, 
            max(mailing_address_1) as mailing_address_1, max(mailing_address_2) as mailing_address_2
            FROM opa_properties_public where mailing_street is not null group by mailing_street
            
        ) as opa
        WHERE opa.num_properties >= 20
        ORDER BY mailing_street
    """
).json()['rows']



In [49]:
df1 = pd.DataFrame(mailing_address_properties)

In [52]:
create_download_link(df1)